## Code for lstm network development. data preparation for lstm can be found in preprocess/seqdata.ipynb

In [1]:
import pandas as pd
import numpy as np
import time
import keras
from math import ceil
import time
import random
import glob,os
import matplotlib.pyplot as plt
import pickle
import pdb
from keras.models import Sequential, Model
from keras.layers import LSTM, Dense, Input, Embedding, Masking
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import MinMaxScaler
from math import sqrt
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from sklearn.base import BaseEstimator
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib


Using TensorFlow backend.
/home/arash/anaconda3/envs/gpusupport/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


### Load Data, not uploaded on github
#### allauxdata columns' corresponding variables:
###### 'Snowy', 'Night', 'One way', 'two way', 'Two way with median', 'Speed Limit_30.0', Speed Limit_40.0', 'Speed Limit_50.0', 'Lane Width_2.5', 'Lane Width_2.75', 'Lane Width_3.0', 'Mean Arrival Rate_530.0', 'Mean Arrival Rate_750.0', 'Mean Arrival Rate_1100.0'

In [2]:
#load data
with open('/home/arash/ProjectVR/cleaneddata/seqdata', 'rb') as f:
    allseqdata = pickle.load(f)

X=allseqdata[0]
o1=allseqdata[1]
o2=allseqdata[2]
o3=allseqdata[3]
dist=allseqdata[4]
y=allseqdata[5]

#load data
with open('/home/arash/ProjectVR/cleaneddata/auxdata', 'rb') as f:
    allauxdata = pickle.load(f) 

### Data Normalization
#### #normalization using min max method: (keras min max not used due to format difference)

In [3]:

#o1,o2,o3: are already bw 0 and 1
maxTraj=[]
minTraj=[]
mindist=[]
for i in range(len(X)):                              #findimg max and min of each feature
    maxTraj.append(max((max(X[i]),max(y[i]))))
    minTraj.append(min((min(X[i]),min(y[i]))))
    mindist.append(min((min(dist[i]),min(dist[i]))))
    
maxXY=max(maxTraj)
maxdist=100
minXY=min(minTraj)
mindist=min(mindist)

Xscaled=[]
yscaled=[]
distscaled=[]
for i in range(len(X)):
    Xscaled.append((X[i]-minXY)/(maxXY-minXY))
    yscaled.append((y[i]-minXY)/(maxXY-minXY))
    distscaled.append((np.array(dist[i])-mindist)/(maxdist-mindist))
    

Xscaled=np.array(Xscaled)
yscaled=np.array(yscaled)
distscaled=np.array(distscaled)

#inverse normalization function using min max method
def invNORM (ypred,y,maxXY,minXY):    # function to calculate RMSE based on inverse normalized
    yinv = np.zeros(y.shape)
    ypredinv=np.zeros(ypred.shape)
    for i in range(len(y)):
        pos = sum (n>=0 for n in y[i])                #position of the last actual value on y, not padded
        yinv[i][:pos]= y[i][:pos] * (maxXY-minXY) + minXY
        ypredinv[i][:pos] = ypred[i][:pos]* (maxXY-minXY) + minXY
    
    return ypredinv, yinv



### padding sequences
#### padding sequences to have same length by adding negative values:


In [4]:
Xscaled = pad_sequences(Xscaled, dtype='float32',value=-0.01)       
o1 = pad_sequences(o1,dtype='float32',value=-0.01)
o2 = pad_sequences(o2,dtype='float32',value=-0.01)
o3 = pad_sequences(o3,dtype='float32',value=-0.01)
distscaled = pad_sequences(distscaled,dtype='float32',value=-0.01)
yscaled = pad_sequences(yscaled, padding='post',dtype='float32',value=-0.01)

### Merging prepared data to create input and output data

In [5]:
inputseq=[]
maxlen = Xscaled.shape[1]
for i in range(Xscaled.shape[0]):
    auxpadded = np.ones(maxlen)*(-0.01)
    auxpadded[:14]=allauxdata[i]     #14 is the number of aux data
    mrg_input=np.transpose(np.vstack((Xscaled[i],o1[i],o2[i],o3[i],distscaled[i],auxpadded)))
    inputseq.append(mrg_input)

inputseq=np.array(inputseq)

### Seperating test set before any modeling

In [6]:
# seperate validation data and test set
tst=0.2                 #% if test data
#val=0.2                 #% if valid data       
tstsize = int(np.floor(len(X) * tst))
#valsize = int(np.floor(len(X) * val))


tstlabel = random.sample(range(0, (inputseq.shape[0]-1)), len(range(tstsize)))
inputseqTEST=inputseq[tstlabel]
ytest=yscaled[tstlabel]

trnlabel=[i for i in range(inputseq.shape[0]) if i not in tstlabel]
inputseqtrain = inputseq[trnlabel]          #Excluding test set
ytrain = yscaled[trnlabel]

### Define Vanilla Model

In [7]:
class VanillaLSTM(BaseEstimator):
    
    def __init__(self, nodes=50, batch_size=32,
                 epochs=200, Llayers=1, features=5, steps_in=83, steps_out=147
                 ):

        self.nodes = nodes
        self.batch_size = batch_size
        self.epochs = epochs
        self.Llayers= Llayers
        self.features = features
        self.steps_in = steps_in
        self.steps_out = steps_out
        self.maxXY=654.7
        self.minXY=650.1


    def create_model(self):
        model = Sequential()
        model.add(Masking(mask_value=-0.01, input_shape=(self.steps_in, self.features)))
        for i in range(self.Llayers-1):
            model.add(LSTM(self.nodes, activation='relu', return_sequences=True))
        model.add(LSTM(self.nodes, activation='relu'))
        model.add(Dense(self.steps_out))
        model.compile(optimizer='adam', loss='mse')
        return model
    
    def fit(self,inputseq,y):
        start = time.time()
        X = inputseq[:,:,:self.features]
        self.model=self.create_model()
        self.model.fit(X, y, epochs = self.epochs, verbose = 0, batch_size = self.batch_size)
        end = time.time()
        print ("Finished Fitting Model. # of Epochs: %d\n Time Taken : %d secs"
               % (self.epochs,end - start))
        return self
    
    def predict(self, inputseq):
        X = inputseq[:,:,:self.features]
        
        return self.model.predict(X)
    
    def score(self,inputseq,y):
        
        X = inputseq[:,:,:self.features]
        ypred = self.predict(inputseq)
        yinv = np.zeros(y.shape)
        ypredinv = np.zeros(ypred.shape)
        for i in range(len(y)):
            pos = sum (n>=0 for n in y[i])                #position of the last actual value on y, not padded
            yinv[i][:pos]= y[i][:pos] * (self.maxXY-self.minXY) + self.minXY
            ypredinv[i][:pos] = ypred[i][:pos]* (self.maxXY-self.minXY) + self.minXY
            
        rmseScore = -sqrt(mean_squared_error(ypredinv, yinv))    #negative of RMSE

        return rmseScore
        

### fitting Vanilla LSTM, using grid search for hypperparameters

In [ ]:
n_features = (inputseq.shape[2]-1)
n_steps_in=inputseq.shape[1]
n_steps_out=ytrain.shape[1]

VLmodel=VanillaLSTM(epochs=1, features=n_features,
                    steps_in=n_steps_in, steps_out=n_steps_out)

In [ ]:
tuned_params = {"Llayers" : [2,3]}#{"nodes" : [10,50,100],"batch_size": [32,64,128] ,"Llayers" : [1,2,3]}

gs = GridSearchCV(VLmodel, tuned_params,cv = 8, refit= True, n_jobs=2)

In [ ]:
grid_result=gs.fit(inputseqtrain,ytrain)

In [ ]:
np.save('/home/arash/ProjectVR/models/GridResults/VLSTM400_8_50_32_L.npy',grid_result.cv_results_)      
#filename: ModelnameEpochs_CV_Nodes_Batch_changingparameters (L:LLayers)

In [ ]:
bestVLSTM = grid_result.best_estimator_

joblib.dump(bestVLSTM, '/home/arash/ProjectVR/models/trainedmodels/VLSTM1.pkl')

In [ ]:
bestVLSTM.score(inputseqTEST,ytest)

In [ ]:
history = bestVLSTM.model.history.history
# plot history
plt.plot(history['loss'], label='train')
#pyplot.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()


## lstm model with aux variables


In [8]:
class AuxLSTM(BaseEstimator):
    
    def __init__(self, nodes=50, batch_size=32,
                 epochs=200, Dlayers=1, Llayers=1, features=5, steps_in=83, steps_out=147
                 ):

        self.nodes = nodes
        self.features = features
        self.steps_in = steps_in
        self.steps_out = steps_out
        self.epochs = epochs
        self.batch_size = batch_size
        self.Dlayers= Dlayers   #number of hidden dense layers
        self.Llayers = Llayers  #number of lstm layers
        self.maxXY=654.7
        self.minXY=650.1


    def create_model(self):
        
        seq_input = Input(shape=(self.steps_in, self.features), dtype='float32', name='seq_input')
        mask = Masking(mask_value=-0.01)(seq_input)
        if self.Llayers==1:
            
            lstm_out = LSTM(self.nodes, activation='relu')(mask)
            
        else:
            lstm_out = LSTM(self.nodes,activation='relu',return_sequences=True)(mask)
            
            for i in range(self.Llayers-2):
                lstm_out = LSTM(self.nodes,activation='relu',return_sequences=True)(lstm_out)
                
            lstm_out = LSTM(self.nodes, activation='relu')(lstm_out)

        #output for lstm, corresponds to 0.2 of loss, used to smooth training and regularization:
        auxiliary_output = Dense(self.steps_out, activation='sigmoid' ,name='aux_output')(lstm_out)   

        auxiliary_input = Input(shape=(14,), name='aux_input')
        x = keras.layers.concatenate([lstm_out, auxiliary_input])

        # We stack a deep densely-connected network on top
        for i in range(self.Dlayers):
            x = Dense(self.nodes, activation='relu')(x)

        # And finally we add the main logistic regression layer
        main_output = Dense(self.steps_out, activation='sigmoid', name='main_output')(x)

        modelaux = Model(inputs=[seq_input, auxiliary_input], outputs=[main_output, auxiliary_output])


        modelaux.compile(optimizer='adam', loss='mse',
                      loss_weights=[1., 0.2])
        return modelaux
    
    def fit(self,inputseq,y):
        start = time.time()
        X = inputseq[:,:,:self.features]
        aux=inputseq[:,:,self.features][:,:14]      
        self.model=self.create_model()
        self.model.fit([X,aux], [y,y], epochs = self.epochs, verbose = 0, batch_size = self.batch_size)
        end = time.time()
        print ("Finished Fitting AuxModel. # of Epochs: %d\n Time Taken : %d secs"
               % (self.epochs,end - start))
        return self
    
    def predict(self, inputseq):
        X = inputseq[:,:,:self.features]
        aux=inputseq[:,:,self.features][:,:14]
        
        return self.model.predict([X,aux])
    
    def score(self,inputseq,y):
        X = inputseq[:,:,:self.features]
        aux = inputseq[:,:,self.features][:,:14]    
        ypred = self.predict(inputseq)[0]        #auxlstm has two identical outputs, 1st in chosen
        yinv = np.zeros(y.shape)
        ypredinv = np.zeros(ypred.shape)
        for i in range(len(y)):
            pos = sum (n>=0 for n in y[i])                #position of the last actual value on y, not padded
            yinv[i][:pos]= y[i][:pos] * (self.maxXY-self.minXY) + self.minXY
            ypredinv[i][:pos] = ypred[i][:pos]* (self.maxXY-self.minXY) + self.minXY
            
        rmseScore = -sqrt(mean_squared_error(ypredinv, yinv))    #negative of RMSE
        return rmseScore
        

In [9]:
n_features = (inputseq.shape[2]-1)
n_steps_in=inputseq.shape[1]
n_steps_out=ytrain.shape[1]

Auxmodel=AuxLSTM(epochs=200, features=n_features,
                    steps_in=n_steps_in, steps_out=n_steps_out)
 

In [12]:
tuned_params = {"Dlayers" : [1,2,3], "Llayers" : [2,3], 'nodes': [50,100]}

gsaux = GridSearchCV(Auxmodel, tuned_params,cv = 8, refit= True, n_jobs=-1)

In [ ]:
gridaux_result=gsaux.fit(inputseqtrain,ytrain)

In [ ]:
np.save('/home/arash/ProjectVR/models/GridResults/AuxLSTM200_8_DLB.npy',gridaux_result.cv_results_)
bestauxLSTM = gridaux_result.best_estimator_

In [ ]:
joblib.dump(bestauxLSTM, '/home/arash/ProjectVR/models/trainedmodels/AuxLSTM1.pkl')
bestauxLSTM.score(inputseqTEST,ytest)

In [ ]:
history = bestauxLSTM.model.history.history
# plot history
plt.plot(history['loss'], label='train')
#pyplot.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()


# Comparing model performac for RMSE of the validation set, 
### all models are trained on same training and validation set, over 200 epochs,
###### (Dense mentioned in models are hidden layers, output dense layer not counted
* **Vanilla 1**: Batch size=32, LSTM units=50,No Masking input, no sample weights for output, two LSTM layer, loss=mse, **RMSE=~71** 

* **Vanilla 2:** Batch size=32, LSTM units=50, Masking input, no sample weights for output, two LSTM layer, loss=mse, **VAL RMSE=~0.29, Test RMSE=0.684**

* **Vanilla 3:** Batch size=32, LSTM units=50, Masking input, -0.01 padded in order to not be considered in RMSE, two LSTM layer, loss=mse, **VAL RMSE=0.302, Test RMSE=0.546**



* **AuxLSTM 1:** Batch size=32, LSTM units=50, Masking input, no sample weights for output, two LSTM layers, , one hidden dense, loss=mse, **VAL RMSE=~0.197, Test RMSE=0.616**

* **AuxLSTM 2:** Batch size=32, LSTM units=50, Masking input, -0.01 padded in order to not be considered in RMSE, two LSTM layers, , one hidden dense, loss=mse, **VAL RMSE=~0.213, Test RMSE=0.538**

In [ ]:
gridaux_result.cv_results_

In [ ]:
besttrain=AuxLSTM(Dlayers=2, Llayers=2, batch_size=32, epochs=1000, features=5, nodes=100,
        steps_in=83, steps_out=147).fit(inputseqtrain,ytrain)

## history = besttrain.model.history.history
# plot history
plt.plot(history['loss'], label='train')
#pyplot.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

In [ ]:
x=besttrain.score(inputseqtrain,ytrain)

In [ ]:
x